# Install and import libraries

In [ ]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [7]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2, prediction_service_pb2_grpc

# Create connection

In [10]:
model_host = "localhost:8500"

channel = grpc.insecure_channel(model_host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Prepare image for prediction

In [29]:
import numpy as np
from urllib import request
from io import BytesIO
from PIL import Image

In [30]:
def get_image_from_url(url, target_size):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    img = img.resize(target_size, Image.NEAREST)
    x = np.array(img, dtype='float32')
    batch = np.expand_dims(x, axis=0)
    return batch

In [83]:
def np_to_ptotobuf(data):
    return tf.make_tensor_proto(data, shape = data.shape)

In [81]:
image_size = (150, 150)
sample_cat = 'https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/manchkin.jpg?raw=true'

In [82]:
X = get_image_from_url(sample_cat, image_size)

# Predicting

In [84]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "cat_breed_model"
pb_request.model_spec.signature_name = "serving_default"
pb_request.inputs["efficientnetb4_input"].CopyFrom(np_to_ptotobuf(X))


In [85]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [86]:
predictions = pb_response.outputs['dense_17'].float_val

# Combining all in one function

In [94]:
class_info = {'Abyssinian': 0,
     'American Bobtail': 1,
     'American Curl': 2,
     'American Shorthair': 3,
     'American Wirehair': 4,
     'Applehead Siamese': 5,
     'Balinese': 6,
     'Bengal': 7,
     'Birman': 8,
     'Bombay': 9,
     'British Shorthair': 10,
     'Burmese': 11,
     'Burmilla': 12,
     'Calico': 13,
     'Canadian Hairless': 14,
     'Chartreux': 15,
     'Chausie': 16,
     'Chinchilla': 17,
     'Cornish Rex': 18,
     'Cymric': 19,
     'Devon Rex': 20,
     'Dilute Calico': 21,
     'Dilute Tortoiseshell': 22,
     'Domestic Long Hair': 23,
     'Domestic Medium Hair': 24,
     'Domestic Short Hair': 25,
     'Egyptian Mau': 26,
     'Exotic Shorthair': 27,
     'Extra-Toes Cat - Hemingway Polydactyl': 28,
     'Havana': 29,
     'Himalayan': 30,
     'Japanese Bobtail': 31,
     'Javanese': 32,
     'Korat': 33,
     'LaPerm': 34,
     'Maine Coon': 35,
     'Manx': 36,
     'Munchkin': 37,
     'Nebelung': 38,
     'No cat': 39,
     'Norwegian Forest Cat': 40,
     'Ocicat': 41,
     'Oriental Long Hair': 42,
     'Oriental Short Hair': 43,
     'Oriental Tabby': 44,
     'Persian': 45,
     'Pixiebob': 46,
     'Ragamuffin': 47,
     'Ragdoll': 48,
     'Russian Blue': 49,
     'Scottish Fold': 50,
     'Selkirk Rex': 51,
     'Siamese': 52,
     'Siberian': 53,
     'Silver': 54,
     'Singapura': 55,
     'Snowshoe': 56,
     'Somali': 57,
     'Sphynx - Hairless Cat': 58,
     'Tabby': 59,
     'Tiger': 60,
     'Tonkinese': 61,
     'Torbie': 62,
     'Tortoiseshell': 63,
     'Turkish Angora': 64,
     'Turkish Van': 65,
     'Tuxedo': 66,
     'York Chocolate': 67}

In [99]:
def get_predictions(url):
    X = get_image_from_url(url, image_size)
    
    pb_request = predict_pb2.PredictRequest()
    pb_request.model_spec.name = "cat_breed_model"
    pb_request.model_spec.signature_name = "serving_default"
    pb_request.inputs["efficientnetb4_input"].CopyFrom(np_to_ptotobuf(X))
    
    pb_response = stub.Predict(pb_request, timeout=20.0)
    
    predictions = pb_response.outputs['dense_17'].float_val
    
    class_predictions = dict(zip(class_info.keys(), predictions))
    return sorted(class_predictions.items(), key=lambda x: x[1], reverse=True)

# Test service

## Munchkin

![image](https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/manchkin.jpg?raw=true)

In [101]:
get_paredictions("https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/manchkin.jpg?raw=true")

[('Burmese', 0.2003280222415924),
 ('Tonkinese', 0.13829274475574493),
 ('Snowshoe', 0.10208065062761307),
 ('Manx', 0.05214511603116989),
 ('British Shorthair', 0.04879487678408623),
 ('Applehead Siamese', 0.047913774847984314),
 ('Exotic Shorthair', 0.044737573713064194),
 ('Siamese', 0.03574749454855919),
 ('Turkish Van', 0.031831610947847366),
 ('Balinese', 0.029916342347860336),
 ('American Bobtail', 0.024295825511217117),
 ('Ragdoll', 0.019599681720137596),
 ('Cornish Rex', 0.018007459118962288),
 ('Japanese Bobtail', 0.016213448718190193),
 ('Scottish Fold', 0.013965381309390068),
 ('Domestic Medium Hair', 0.013827316462993622),
 ('Extra-Toes Cat - Hemingway Polydactyl', 0.013286962173879147),
 ('American Shorthair', 0.012148640118539333),
 ('Oriental Short Hair', 0.011581959202885628),
 ('Oriental Tabby', 0.008972340263426304),
 ('Dilute Calico', 0.008972052484750748),
 ('Egyptian Mau', 0.008074469864368439),
 ('Tiger', 0.008028769865632057),
 ('Himalayan', 0.007902013137936592

## Chausie

![image](https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/chausie.jpg?raw=true)

In [103]:
get_predictions("https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/chausie.jpg?raw=true")

[('Abyssinian', 0.4517592787742615),
 ('Cornish Rex', 0.07613372057676315),
 ('Oriental Short Hair', 0.055158115923404694),
 ('Sphynx - Hairless Cat', 0.046744052320718765),
 ('Bengal', 0.04664374515414238),
 ('Devon Rex', 0.039809685200452805),
 ('Ocicat', 0.038995832204818726),
 ('Burmese', 0.02415214665234089),
 ('Extra-Toes Cat - Hemingway Polydactyl', 0.019073331728577614),
 ('American Shorthair', 0.018102427944540977),
 ('Tonkinese', 0.015269175171852112),
 ('Chausie', 0.015097995288670063),
 ('Egyptian Mau', 0.013392817229032516),
 ('Manx', 0.011986064724624157),
 ('Snowshoe', 0.00967077910900116),
 ('Exotic Shorthair', 0.008334281854331493),
 ('Oriental Tabby', 0.008212610147893429),
 ('Munchkin', 0.007630824577063322),
 ('Tortoiseshell', 0.006851544138044119),
 ('Siamese', 0.006692946422845125),
 ('Javanese', 0.00643542967736721),
 ('Torbie', 0.006184724159538746),
 ('American Bobtail', 0.006046625319868326),
 ('Calico', 0.005115384701639414),
 ('Domestic Short Hair', 0.004848

## Burmilla

![image](https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/burmila.jpg?raw=true)

In [104]:
get_predictions("https://github.com/rzabolotin/ml_zoomcamp_2022_project_2/blob/main/static/burmila.jpg?raw=true")

[('Turkish Van', 0.20681680738925934),
 ('Snowshoe', 0.15892663598060608),
 ('American Bobtail', 0.08314710855484009),
 ('Oriental Short Hair', 0.06432677805423737),
 ('Siamese', 0.06081867590546608),
 ('Scottish Fold', 0.0315106175839901),
 ('British Shorthair', 0.029953358694911003),
 ('Manx', 0.02979104034602642),
 ('Oriental Tabby', 0.029013564810156822),
 ('Dilute Calico', 0.020816698670387268),
 ('Egyptian Mau', 0.019914938136935234),
 ('Cornish Rex', 0.019769614562392235),
 ('American Shorthair', 0.01723439246416092),
 ('Japanese Bobtail', 0.016291745007038116),
 ('Extra-Toes Cat - Hemingway Polydactyl', 0.015309331938624382),
 ('Silver', 0.010912129655480385),
 ('Devon Rex', 0.010711533017456532),
 ('Domestic Medium Hair', 0.01055151503533125),
 ('American Curl', 0.010132398456335068),
 ('Tabby', 0.010079162195324898),
 ('Munchkin', 0.009996985085308552),
 ('Applehead Siamese', 0.009885073639452457),
 ('Domestic Short Hair', 0.009864643216133118),
 ('Javanese', 0.00955805554986

# Conclussion

So, unfornunatelly my model gives not so good predictions.  
Maybe I need to spend more time to optimize it.  

But our deployment works, and we will use this model to create a service.